In [ ]:
# LightGBM + Optuna 튜닝 (callbacks 기반, verbose 완전 제거)
# 대상: 타워램프 상태 분류

import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import early_stopping, log_evaluation
from optuna.visualization import plot_optimization_history, plot_param_importances

# 데이터 로드
df = pd.read_csv('led_features_20250723_172434.csv')
X = df.drop(columns=['label', 'image_name', 'label_name'])
y = df['label']

# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Optuna objective 함수
def objective(trial):
    params = {
        'n_estimators': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 5.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 5.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'objective': 'multiclass',
        'num_class': len(set(y)),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(50), log_evaluation(0)]
    )
    preds = model.predict(X_val)
    acc = accuracy_score(y_val, preds)
    return 1.0 - acc

# Optuna 스터디 생성 및 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# 결과 출력
print("Best parameters:", study.best_params)
print("Best accuracy:", 1.0 - study.best_value)

# 시각화
plot_optimization_history(study).show()
plot_param_importances(study).show()
